<a href="https://colab.research.google.com/github/UITTrinhQuangTruong/CS114.K21/blob/master/%C4%90%E1%BB%93_%C3%A1n_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Đổi tên hàng loạt

In [6]:
#405 Cobaoluc
#412 Khongbaoluc

#Tiền xử lý Dữ liệu

## Import các thư viện cần thiết

In [7]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from skimage.feature import hog
from sklearn.cluster import KMeans
from os import listdir
from os.path import isfile, join

##Trích xuất Optical flow bằng thuật toán TVL1 có trong OpenCV
 

### Định nghĩa về Optical flow
Optical Flow là khái niệm chỉ sự chuyển động tương đối của các điểm trên bề mặt một đối tượng, vật thể nào đó gây ra, dưới góc quan sát của một điểm mốc (mắt, camera…).

Dễ hiểu là điểm khác biệt giữa 2 khung hình do vật thể di chuyển gây ra.

![Hình ảnh mô tả Optical flow của các xe](https://opencv-python-tutroals.readthedocs.io/en/latest/_images/opticalflow_lk.jpg)

###Các bước thực hiện
B1: Định nghĩa đối tượng bằng hàm DualTVL1OpticalFlow_create()

B2: Tính flowx, flowy của 2 frame gần nhau bằng hàm calc. Với flowx là Optical Flow theo hướng x và tương tự với y

B3: Tính mag = sqrt(flowx + flowy)

B4: Normalize mag về ảnh hsv

B5: Chuyển ảnh về định dạng Grayscale

In [8]:
optical_flow = cv2.optflow_DualTVL1OpticalFlow.create()
def OpticalFlow(prvs, next, hsv):
    
    flow = optical_flow.calc(prvs, next, None)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    nom = np.zeros((160,120))
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)

    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    return gray

##Trích xuất vùng chuyển động
Từ ảnh Optimal Flow vừa tìm được ta trích xuất ra vùng có sự chuyển động


###Phát hiện cạnh với thuật toán Canny
![Cận trên và dứoi](https://docs.opencv.org/trunk/hysteresis.jpg)

Thuật toán Canny tham khảo thêm [tại đây](https://docs.opencv.org/trunk/da/d22/tutorial_py_canny.html)

Hàm tìm maxVal và minVal tham khảo thêm [tại đây](https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/)

In [9]:
def CvCanny(matrix):
  v = np.median(matrix)
  sigma = 0.33
  lower = int(max(0, (1.0 - sigma) * v))
  upper = int(min(255, (1.0 + sigma) * v))
  canny = cv2.Canny(matrix, lower, upper)
  return canny

###Trích xuất vùng chuyển động

In [16]:
def MRM(prvs, next, hsv):
    #Trích xuất dòng quang từ 2 frame gần nhau
    gray = OpticalFlow(prvs, next, hsv)
    '''if s == 3:
    flowx = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
    cv2_imshow(flowx)
    flowy = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
    cv2_imshow(flowy)'''
    '''Tăng độ nét cho ảnh dòng quang
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpen = cv2.filter2D(hsv, -1, kernel)
    canny1 = CvCanny(sharpen)'''
    #Phát hiện cạnh
    canny = CvCanny(gray)
    #gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    #cv2_imshow(gray)    
    '''gray1 = cv2.filter2D(gray, -1, kernel)[...,::-1]
    gray = cv2.ximgproc.guidedFilter(gray, gray1, radius=2, eps=0.004, dDepth=-1)[...,::-1]
    canny3 = CvCanny(gray)
    cv2_imshow(gray)'''
    #Đóng các nét hở và lấp đầy
    kernel = np.ones((5,5),np.uint8)
    closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
    '''h, w = closing.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    im_floodfill = closing.copy()
    cv2.floodFill(im_floodfill, mask, (0,0), 255);
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    floodfill = closing | im_floodfill'''

    #Lấp đầy những lỗ hổng trong vùng chuyển động
    contour,hier = cv2.findContours(closing,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
        cv2.drawContours(closing,[cnt],0,255,-1)
    
    #Xóa bỏ những nét thừa
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    
    return opening

#Trích xuất đặc trưng


##Vùng quan tâm

Xem vùng chuyển động như là một vùng quan tâm (ROI), áp dụng ROI vào ảnh xám

In [11]:
def apply_roi(img, roi):
  #thresh, roi = cv2.threshold(roi, thresh=128, maxval=1, type=cv2.THRESH_BINARY)
  new_img = img * roi
  return new_img

##Trích xuất đặc trưng

In [12]:
def extract_features(path, label, descriptor_list, labels):
    #Lấy danh sách các clip trong path 
    videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    for videoPath in videoPaths:

        #Đọc video
        cap = cv2.VideoCapture(videoPath)
        videoPaths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]

        ret, frame1 = cap.read()
        if not ret:
          break;
        #Thay đổi kích thước frame
        frame1 = cv2.resize(frame1, (160, 120))
        prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255
        s = 1
        while(cap.isOpened()):
            ret, frame2 = cap.read()

            if not ret:
                break;

            if s % 6 == 0:
                frame2 = cv2.resize(frame2, (160, 120))
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                mrm = MRM(prvs, next, hsv)

                after_roi = apply_roi(next, mrm)
                descriptor = hog(after_roi, orientations=12, pixels_per_cell=(4, 4), cells_per_block=(2,2), block_norm='L2-Hys', visualize=False)
                descriptor_list.append(descriptor)
                labels.append(label)
                prvs = next
            s += 1

        print(videoPath + "---done--")


In [13]:
labels = []
descriptor_list = []

In [17]:
path = "/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc"
extract_features(path, 0, descriptor_list, labels)
path = "/content/drive/My Drive/Machine Lol/Data/Phu/Khongbaoluc"
extract_features(path, 1, descriptor_list, labels)

/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/272.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/273.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/274.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/275.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/276.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/277.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/278.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/279.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/280.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/281.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/282.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/283.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/284.mp4---done--
/content/drive/My Drive/Machine Lol/Data/Phu/Cobaoluc/285.mp4---done--
/conte

In [19]:
print(len(descriptor_list))
print(len(labels))

2131
2131


###Lưu lại đặc trưng cho lần train tiếp theo

In [27]:
import hickle as hck

hck.dump(descriptor_list, '/content/drive/My Drive/Machine Lol/backup/data1.hkl', mode='w')

hck.dump(labels, '/content/drive/My Drive/Machine Lol/backup/labels1.hkl', mode='w')

#Training model

###Load Data Backup(Nếu có)

In [28]:
descriptor_list = hck.load('/content/drive/My Drive/Machine Lol/backup/data1.hkl')
labels = hck.load('/content/drive/My Drive/Machine Lol/backup/labels1.hkl')

print(len(descriptor_list))
print(len(labels))

2131


##Chia tỉ lệ Train|Test
Tỉ lệ giữa data train và data test là 0.8|0.2

In [22]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(descriptor_list, labels,test_size =0.2, random_state=42)

##Model LogisticRegression

In [29]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

lr = LogisticRegression()

lr.fit(trainX, trainY)


print(lr.score(trainX, trainY))
print(lr.score(testX, testY))

predY = lr.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

1.0
0.9578454332552693
(0.9772151898734177, 0.8200000000000001, 0.8785858713509415, None)


##Model GaussianNB

In [31]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()


gnb.fit(trainX, trainY)

print(gnb.score(trainX, trainY))
print(gnb.score(testX, testY))

predY = gnb.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

0.9536384976525821
0.9250585480093677
(0.8353682170542636, 0.7754111405835544, 0.8012798138452589, None)


##Model RandomForestClassifier

In [32]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 10, random_state= 0)

rfc.fit(trainX, trainY)

print(rfc.score(trainX, trainY))
print(rfc.score(testX, testY))

predY = rfc.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

0.9865023474178404
0.9086651053864169
(0.953125, 0.61, 0.6557377049180327, None)


##Model SVM

In [33]:
from sklearn import svm

lsvc = svm.SVC()
lsvc.fit(trainX, trainY)

print(lsvc.score(trainX, trainY))
print(lsvc.score(testX, testY))

predY = lsvc.predict(testX)

print(precision_recall_fscore_support(testY, predY, average='macro'))

0.9917840375586855
0.9250585480093677
(0.960880195599022, 0.6799999999999999, 0.744349648256249, None)


#Ứng dụng